<img src="https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/agods/nyp_ago_logo.png" width='400'/>

# Introduction to ordinary least squares (OLS) Regression



In [ ]:
# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
%matplotlib inline

# Survey Data
We will use this [simple survey data](https://stats.idre.ucla.edu/wp-content/uploads/2016/02/p054.txt) to demonstrate a few basic features of ***statsmodels*** and ***seaborn*** and how they might be used in a data science workflow for regression.

The dataset is simply the results of a survey where the question responses are all numeric.  This leads to 6 numeric independent variable (predictor or feature) fields and 1 numeric dependent variable (response or target) field.  The predictors are labeled ***X<sub>i</sub>*** and the response is labeled ***Y***.

Load the dataset in using ***pandas*** and take a look at it.  Here we use [***pandas.read_table***](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html) to load the data.

In [ ]:
# Load data
df = pd.read_table('data/p054.txt')


In [ ]:
# Take a look at the datatypes
#Add code

In [ ]:
# Take a look at the first few rows
#Add code

If we look at the column names, we will notice the trailing whitespace problem.

In [ ]:
#Add code

We can remove this by calling map on the columns list and stripping the whitespace with strip.  The ***map*** function is applied to Series objects, whereas the ***apply*** and ***applymap*** functions are called on Dataframes.

In [ ]:
df.columns = df.columns.map(str.strip)
df.columns

In [ ]:
# How many rows and columns does the dataset have?
#Add code


### Visualizing with Seaborn
We see that the data has 30 responses with 7 fields (6 independent, 1 dependent) each.  Let's use pandas to check out the correlations between the different variables.

In [ ]:
# View the correlations
#Add code

In [ ]:
# Use seaborn heatmap for a better visualisation
sns.heatmap(df.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);

# more cmaps: https://matplotlib.org/examples/color/colormaps_reference.html

### Correlation and Multicollinearity
We notice that some of the variables are highly correlated. When 2 predictor variables (i.e. features) are highly correlated this is called [multicollinearity](https://en.wikipedia.org/wiki/Multicollinearity) and it is something we want to watch out for as it can destabilize our model.  In the extreme case, when 2 features are perfectly correlated then there is absolutely nothing gained by making both variables part of our regression.

The other takeaway from this table is that some of our features are highly correlated with our ***target variable Y***.  This is a good thing, it means that these are the variables that we most likely want to include as part of our model as they explain a large amount of the variance in the target variable (correlation=R, variance_explained=R<sup>2</sup>).

Let's try to visualize these correlations all together by using the [***seaborn pairplot***](http://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.pairplot.html) function.

> What do you notice?
> Almost all correlations are positive, somewhat normal distributions, etc.

In [ ]:
# Plot all of the variable-to-variable relations as scatterplots
sns.pairplot(df, height=1.2, aspect=1.5);

### Ordinary Least Squares Regression with Statsmodels
Now that we have a feel for our data, let's try to build a basic regression model.  

#### Statsmodels
We are going to use the [**`statsmodels`**](http://statsmodels.sourceforge.net/) library first.  `statsmodels` is a Python package for implementing [**linear models**](https://en.wikipedia.org/wiki/Linear_model), of which **Linear Regression** is one.  It has a bunch of nice features for evaluating and executing such models.

#### Modeling with Statsmodels
There are 2 main ways you can generate models with stats models:
- Via the `statsmodels.api` package
- Via the `statsmodels.formula.api` package

For both approaches, you will need somewhere to use the [R formula](http://science.nature.nps.gov/im/datamgmt/statistics/r/formulas/) styles formulas for defining the relationship between target variable and feature variables in your model.  ***Statsmodels*** uses [***patsy***](http://patsy.readthedocs.org/en/latest/) to convert this syntax into the proper data matrices for input into its linear models under the covers.  There are a variety of interactions and functions of variables that you can incorporate with this syntax, so feel free to check out the docs.

**Model 1 - 6 features**

Here we'll just start by defining a regression model that takes as its inputs each of the 6 predictor variables.  The other parameter of course is the data that the model is to be built from, our pandas dataframe.

This first model fitting is done for you, it fits a multiple linear regression model of the following form:

$$
\hat{Y} = \beta_0 + \beta_1X_1 + \beta_2X_2 + \beta_3X_3 + \beta_4X_4 + \beta_5X_5 + \beta_6X_6
$$

##### `statsmodels.api`
To use this method, you need to generate a **matrix** of **features**, **`X`** and a **vector** of **targets**, **`y`** where each row represents a single **observation**.  In statsmodels, you can do this with a call to **`patsy.dmatrices`**:


In [ ]:
# Create your feature matrix (X) and target vector (y)
y, X = patsy.dmatrices('Y ~ X1 + X2 + X3 + X4 + X5 + X6', data=df, return_type="dataframe")

In [ ]:
#display y first 5 rows
#Add code

In [ ]:
#display X fisrt 5 rows
#Add code

In [ ]:
# Create your model
model = sm.OLS(y, X)

# Fit your model to your training set
fit = model.fit()

# Print summary statistics of the model's performance
fit.summary()

## Regression Statistics

***R<sup>2</sup>*** is the square of the correlation coefficient and represents the estimated percentage of the variance in our target variable ***Y*** that can be explained by our regression model.  

***Adjusted R<sup>2</sup>*** also penalizes for things such as large coefficients and extra variables to try and limit ***overfitting*** so it is often a better measure of model efficacy.

The middle table provides the **coefficients** that regression has found, along with the **standard error** for each coefficient. This defines our model, i.e. these are the model parameters that our algorithm was seeking to determine.  


**Model 2 - 3 features**

Putting it all together, the final column returns a **95% Confidence Interval** for the value of each coefficient.

Given these stats, lets remove the highest 3 P-values (low p-values are optimal) from our regression model, from ***X<sub>2</sub>***, ***X<sub>4</sub>***, and ***X<sub>5</sub>*** and see how our model performs now:

In [ ]:
# Define the model without X2, X4, and X5
lm2 = smf.ols('Y ~ X1 + X3 + X6', data=df)

# Fit the model
fit2 = lm2.fit()

# Check out the results
fit2.summary()

You should see our **Adjusted R<sup>2</sup>** has increased, and our P-values are lower so we likely have a better model.

**Model 3 - 2 features**

Let's just try removing ***X<sub>6</sub>*** to see if that might improve our model a little bit more.

In [ ]:
# Define the model removing X6 this time
lm3 = smf.ols('Y ~ X1 + X3', data=df)

# Fit the model
fit3 = lm3.fit()

# Check out the results
fit3.summary()

Notice both **R<sup>2</sup>** coefficients decreased so let's stick with model 2.


# Regression with sklearn
Statsmodels has decent functionality for linear models, and is great for statistical summaries. But, scikit-learn has more modeling options for all sorts of algorithms as well as data preparation so it is commonly used too.

### Regression with sklearn
Before we jump into some of the additional features of sklearn, let's try to repeat our basic survey example using sklearn's built in **LinearRegression**.

You should still have your Dataframe loaded from earlier.  Let's try repeating some of the different models we tried earlier with statsmodel.

**Model 1 - 6 features**

Here's the first model with 6 features.

In [ ]:
# Create an empty model
lr = LinearRegression()

# Choose the predictor variables, here all but the first which is the response variable
# This model is analogous to the Y ~ X1 + X2 + X3 + X4 + X5 + X6 model
X = df.iloc[:, 1:]

# Choose the response variable(s)
y = df.iloc[:, 0]


In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Fit the model to the full dataset
lr.fit(X, y)

# Print out the R^2 for the model against the full dataset
lr.score(X,y)

You'll notice that this is the same **R<sup>2</sup>** value that was reported for the model 1 using statsmodels above.

**Model 2 - 3 features**

Let's quickly run the best model from earlier (***X1***, ***X3***, and ***X6***) and see how it performs.

In [ ]:
# Create an empty model
lr1 = LinearRegression()

# Choose the predictor variables, here all but the first which is the response variable
# This model is analogous to the Y ~ X1 + X3 + X6 model
X = df[['X1', 'X3', 'X6']]

# Choose the response variable(s)
y = df['Y']

# Fit the model to the full dataset
lr1.fit(X, y)

# Print out the R^2 for the model against the full dataset
lr1.score(X, y)


In [ ]:
adjusted_r_squared = 1 - (1-lr1.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)
adjusted_r_squared

Notice that **R<sup>2</sup>** and adjusted **R<sup>2</sup>** are similar to what you obtained using statsmodels.

# Exercise 1: Car Price Predictor Dataset
For these exercises we'll be exploring the auto data available [here](https://archive.ics.uci.edu/ml/datasets/Automobile).  The goal is to be able to predict auto price from the dataset.

## Data Exploration
Use pandas `read_csv()` to load the data into a dataframe and then call `head()` to make sure everything looks good.

In [ ]:
# read in the car dataset
df=pd.read_csv('data/imports-85.data',header=None)

columns= ['symboling','normalized-losses','make','fuel-type','aspiration','num-of-doors','body-style','drive-wheels','engine-location','wheel-base','length','width','height','curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore','stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']
df.columns=columns
# Use head to view the first few rows
df.head()


Use [`shape`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html#pandas.DataFrame.shape) to check out how many rows and columns the dataframe has.

In [ ]:
# How many rows and columns do we have?
print(df.shape)

Use [`info()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html#pandas.DataFrame.info) to get a summary of the dataframe and its datatypes

In [ ]:
# Let's examine the datatypes
df.info()

**Lets keep numeric data from wheel-base onwards to price**
- Create a list of columns to keep
- Select out only those columns from the dataframe and reassign the dataframe to that selection
- Use `head()` & `info()` to make sure everything worked as expected

In [ ]:
# Create a list of columns to keep

subset=['wheel-base','length','width','height','curb-weight',
        'engine-size','bore','stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']
cars=df.loc[:,subset]

cars.info()


It looks like some of our features (bore, stroke, horsepower, peak-rpm and price) are listed as objects and not float64 or in64.

Run cars.head(10), and we will see why!  

In [ ]:
# check data head
cars.head(10)

In [ ]:
# print lists of observations where price is ?
cars[cars['price']=='?']

In [ ]:
# print entries with ?
objects=['bore','stroke','horsepower','peak-rpm','price']
for o in objects:
    print('%s: %d' % (o, len(cars[cars[o]=='?'])))
    display(cars[cars[o]=='?'])


In [ ]:
# replace "?" data in order to turn our features into numerics

objects=['bore','stroke','horsepower','peak-rpm','price']
for o in objects:
    cars[o]=cars[o].replace('?',np.nan)
    cars[o]=cars[o].astype(float)

cars.info()

We can see from the above output, that only a few entries were unknown in the first place.   

To keep things simple for now, let's just go ahead and drop the entries that were unknown.


In [ ]:
cars.head(10)

In [ ]:
#cars['price'].isnull().sum()

In [ ]:
#cars['price'].isnull()

In [ ]:
#cars['price'].isnull().any()

In [ ]:
cars=cars.dropna()
len(cars)

Before we begin modeling, use the [`corr()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html#pandas.DataFrame.corr) function to get a feel for the correlations among the different variables, especially with regard to 'price'.

In [ ]:
cars.corr()

Take a look at only the 'price' column of the correlations and order it in descending order wih [`sort_values()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html#pandas.DataFrame.sort_values)

In [ ]:
# Try using seaborn heatmap for a better view
plt.figure(figsize = (16,5))

sns.heatmap(cars.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);

In [ ]:
# Get the correlations with 'price' sorted in descending order
cars.corr()['price'].sort_values(ascending=False)

You should now have a better feel for which variables might be most valuable for your model.
> Q:  Do correlations provide the 'entire picture' of what is happening with our model?

> A:  No. It can give us an idea but corrs will only provide the relationship with the response variable (all other factors being held constant)

Now use ***seaborn's*** `pairplot()` function to visualise these correlations for the variables.

In [ ]:
#Let's try visualizing some of these pairwise correlations with seaborn
sns.pairplot(cars[['price','engine-size', 'curb-weight', 'horsepower', 'width', 'length', 'wheel-base','bore']])

## Exercise: Modeling with statsmodels
Let's try some exploration with statsmodels.  As a first model, try creating an ordinary least squares model with statsmodels by incorporating all of the variables that had at least a .10 absolute value of correlation.
- Create your model with the `ols()` function with the appropriate **R Formula** syntax and your dataframe
- Fit the model
- Print the fit summary to check out the results

In [ ]:
# Let's jump right in and try a model with statsmodels using all variables above .10 correlation
# lsm = smf.ols('price~ ....', data = cars)

# You might have issues with one of the features due to dash.. Fix variable names
cars.rename(
    inplace=True,
    columns={
        "curb-weight": "curb_weight",
        'engine-size': 'engine_size',
        'wheel-base': 'wheel_base'
    })

# this also works
#cars.rename(columns=lambda name: name.replace("-", "_"), inplace=True)

cars.head()

In [ ]:
# fit and summarize
lsm = smf.ols('price~ engine_size + curb_weight +horsepower + width + length + wheel_base + bore + height', data = cars)
fit1 = lsm.fit()
fit1.summary()

#### Seaborn for Exploring Distributions
Your **R<sup>2</sup>** should be .834, which is not too bad. That means we can explain about 83.4% of the variance in price with this model.  


## Improving Model ##

If you remember from your lecture, we made assumptions about Linear Regression, one of which being: normal distribution of the predictor variable. Perhaps you have noticed from our pairplot above that our 'price' variable is skewed. Transform the y variable and rerun your OLS model. Are there any other variables we should transform?


In [ ]:
# take log of price and graph

cars['log_price']=np.log(cars.price)
# looks better
cars.log_price.hist()

In [ ]:
# refit and summarize
lsm = smf.ols('log_price~ engine_size + curb_weight +horsepower + width + length + wheel_base + bore + height', data = cars)
fit2 = lsm.fit()
fit2.summary()

You should see a slightly better **R<sup>2</sup>**